### Word2Vec

In [60]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import re
import pandas as pd
import json

In [99]:
RE_WORD = r'\w+'
re_word = re.compile(RE_WORD, re.U)

DATA_SET = [
    "data/frankenstein.txt",
    "data/moby_dick.txt",
    "data/freud.txt", 
    "data/wiki.txt", 
    "data/middlemarch.txt", 
    "data/pride_and_prejudice.txt", 
    "data/alice_in_wonderland.txt",
    "data/bleakhouse.txt",
    "data/crime_and_punishment.txt",
    "data/room_with_a_view.txt"
]

def extract_vocab(paths: list[str]) -> dict[str, int]:
    vocab = dict()
    for path in paths:
        with open(path, encoding="utf-8") as file:
            for word in (word.lower() for word in re.findall(re_word, file.read())):
                if word not in vocab:
                    vocab[word] = 1
                else:
                    vocab[word] = vocab[word] + 1

    words = (word for word, count in vocab.items() if count > 8)
    return { word:idx for idx, word in enumerate(words) }

def tokenize(text: str, vocab: dict[str, int]) -> list[str]:
    return [word.lower() for word in re.findall(re_word, text) if word in vocab]

def load_data(path: str) -> str:
    with open(path, encoding="utf-8") as file:
        text = file.read()
        return text

vocab = extract_vocab(DATA_SET)

print(len(vocab))

11024


In [100]:
def batch_to_one_hot(batch: list[list[int]], vocab_size: int) -> torch.Tensor:
    t = torch.zeros((len(batch), vocab_size), dtype=torch.float32)

    for i, indeces in enumerate(batch):
        t[i, indeces] = 1.0
    
    return t

def tokens_to_indeces(tokens: list[str], vocab: dict[str, int]) -> list[int]:
    return [vocab[token] for token in tokens]

def gen_text_to_tensors(text: str, vocab: dict[str, int], window_size: int, batch_size: int):
    tokens = tokenize(text, vocab)
    indeces = tokens_to_indeces(tokens, vocab)
    half_window = window_size // 2
    

    words = []
    context = []
    for i in range(half_window, len(indeces) - half_window):
        words.append([indeces[i]])
        context.append(indeces[i-half_window:i] + indeces[i+1:i+1+half_window])

        if (i % batch_size == 0):
            yield batch_to_one_hot(words, len(vocab)), batch_to_one_hot(context, len(vocab))
            words = []
            context = []
        
    yield batch_to_one_hot(words, len(vocab)), batch_to_one_hot(context, len(vocab))


In [121]:
VOCAB_SIZE = len(vocab)
VECTOR_SIZE = 100

model = nn.Sequential(
    nn.Linear(VOCAB_SIZE, VECTOR_SIZE),
    nn.Linear(VECTOR_SIZE, VOCAB_SIZE),
    nn.Softmax()
)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [122]:
epochs = 10

for epoch in range(epochs):
    for data in DATA_SET:
        text = load_data(data)
        
        for Y, X in gen_text_to_tensors(text, vocab, window_size=5, batch_size=1024):
            outputs = model(X.float())
            loss = loss_fn(outputs, Y.float())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()


In [124]:
embeddings = torch.transpose(list(model.parameters())[0].detach().clone(), 0, 1)

wordsim353 = pd.read_csv("wordsim353/combined.csv")

not_in_vocab = set()
wordsim353['Word 1'] = wordsim353['Word 1'].apply(lambda w: w.lower())
wordsim353['Word 2'] = wordsim353['Word 2'].apply(lambda w: w.lower())
wordsim353['Human (mean)'] = wordsim353['Human (mean)'].apply(lambda m: m / 5 - 1)

count = 0
error = 0
not_in_vocab = set()
for _, row in wordsim353.iterrows():
    w1 = row['Word 1']
    w2 = row['Word 2']
    human = row['Human (mean)']

    if w1 not in vocab:
        not_in_vocab.add(w1)
        continue
    if w2 not in vocab:
        not_in_vocab.add(w2)
        continue

    sim = F.cosine_similarity(embeddings[vocab[w1]].unsqueeze(0), embeddings[vocab[w2]].unsqueeze(0)).tolist()[0]
    error += (human - sim) ** 2
    count += 1
    print(f'{human:.3f}, {sim:.3f}, {w1}, {w2}')


print("MSE: ", error / count)

0.354, -0.620, love, sex
0.470, 0.807, tiger, cat
1.000, 1.000, tiger, tiger
0.492, 0.857, book, paper
0.524, 0.474, computer, keyboard
0.516, 0.685, computer, internet
0.500, 0.515, telephone, communication
0.354, 0.667, television, radio
0.484, -0.293, media, radio
0.370, 0.142, drug, abuse
0.238, 0.593, bread, butter
0.400, -0.281, doctor, nurse
0.324, 0.661, professor, doctor
0.362, 0.770, student, professor
-0.076, 0.817, smart, student
0.162, -0.204, smart, stupid
0.416, -0.092, company, stock
0.616, -0.462, stock, market
-0.676, 0.385, stock, phone
-0.638, 0.429, stock, egg
-0.254, -0.410, stock, live
-0.816, -0.106, stock, life
0.492, 0.308, book, library
0.624, 0.877, bank, money
0.546, 0.436, wood, forest
0.830, 0.676, money, cash
0.716, -0.311, king, queen
0.692, -0.047, jerusalem, israel
0.530, -0.035, jerusalem, palestinian
-0.676, -0.588, holy, sex
0.724, 0.105, maradona, football
0.806, 0.287, football, soccer
0.326, 0.934, football, tennis
0.512, -0.824, tennis, racket


In [125]:
with open("out/embeds.json", "+w", encoding="utf-8") as file:
    file.write(json.dumps({ word:embeddings[idx].tolist() for word, idx in vocab.items() }))